In [529]:
%pip install redshift_connector
%pip install python-dotenv
%pip install matplotlib
%pip install scipy
%pip install -U scikit-learn
%pip install seaborn
%pip install itertools
%pip install numpy
%pip install pandas
%pip install matplotlib-venn
%pip install scipy.stats

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the require

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.stats as stats
import seaborn as sns; sns.set()
import warnings
from sklearn.metrics import davies_bouldin_score, silhouette_score, silhouette_samples
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")
%matplotlib inline

In [7]:
#visualizando como tá ficando
df.describe().apply(lambda s: s.apply('{0:.3f}'.format))

,id_contrato,aging,ultimo_pagamento,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,receita,seq_meses,transacoes
count,2266350.000,2266350.000,2266350.000,2266350.000,2266350.000,2266350.000,2266350.000,2266350.000,2266350.000,2266350.000,2266350.000,2266350.000,2266350.000
mean,23339669.800,26.413,19.291,0.745,0.785,0.820,0.841,0.882,0.894,0.741,337.997,5.708,5.714
std,5869883.026,28.458,15.578,0.436,0.411,0.384,0.366,0.323,0.307,0.438,151.419,1.789,1.790
min,49.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000
25%,20537304.500,7.000,9.000,0.000,1.000,1.000,1.000,1.000,1.000,0.000,238.000,5.000,5.000
50%,25781109.500,12.000,17.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,353.940,6.000,6.000
75%,27614286.750,38.000,27.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,415.930,7.000,7.000
max,29204336.000,129.000,205.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1244.920,7.000,16.000


In [8]:
#drop das colunas de transacoes nos meses
df2 = df.drop(columns=[ '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01','2023-06-01','2023-07-01'])
display(df2)

,id_contrato,aging,ultimo_pagamento,receita,seq_meses,transacoes
0,12495686,84,8,555.93,7,7
1,26132537,11,23,277.83,7,7
2,26141204,11,29,179.34,6,6
3,28546838,4,2,199.96,4,4
4,29065442,1,18,59.99,1,1
...,...,...,...,...,...,...
2266345,29025344,1,24,49.99,1,1
2266346,29007653,1,27,49.99,1,1
2266347,28862852,2,49,49.99,1,1
2266348,28894834,2,45,49.99,1,1


In [9]:
#Criando o grupo dos aderidos <- Otimizar esse mapeamento de grupos manualmente
df3 = df2
df3['adesao'] = np.where(df3['transacoes'] <= 1, 'adesao','outros')
print(df3)

         id_contrato  aging  ultimo_pagamento  receita  seq_meses  transacoes  \
0           12495686     84                 8   555.93          7           7   
1           26132537     11                23   277.83          7           7   
2           26141204     11                29   179.34          6           6   
3           28546838      4                 2   199.96          4           4   
4           29065442      1                18    59.99          1           1   
...              ...    ...               ...      ...        ...         ...   
2266345     29025344      1                24    49.99          1           1   
2266346     29007653      1                27    49.99          1           1   
2266347     28862852      2                49    49.99          1           1   
2266348     28894834      2                45    49.99          1           1   
2266349     29025702      1                24    49.99          1           1   

         adesao  
0        

In [10]:
#criando o subset final com aqueles que nao foram adesao / o df2 tem todos os registros
df3 = df3[(df3["adesao"] != "adesao")] # & (df3["potent_churn"] != "churn") & (df3["outliers_transacoes"] != "outlier") ]
print(df3)

         id_contrato  aging  ultimo_pagamento  receita  seq_meses  transacoes  \
0           12495686     84                 8   555.93          7           7   
1           26132537     11                23   277.83          7           7   
2           26141204     11                29   179.34          6           6   
3           28546838      4                 2   199.96          4           4   
5           18937705     51                27   413.94          6           6   
...              ...    ...               ...      ...        ...         ...   
2266282     28786817      2                27   119.98          2           2   
2266284     28780924      2                29    99.98          2           2   
2266289     28830525      2                23    99.98          2           2   
2266295     28774317      2                27    99.98          2           2   
2266299     28871065      2                18    99.98          2           2   

         adesao  
0        

In [11]:
#Criando as categorias de valores de Recência pra base toda

# lista de condicoes
condicoes = [
    (df3['ultimo_pagamento'] <= 30),
    (df3['ultimo_pagamento'] > 30) & (df3['ultimo_pagamento'] <= 45),
    (df3['ultimo_pagamento'] > 45) & (df3['ultimo_pagamento'] <= 60),
    (df3['ultimo_pagamento'] > 60) & (df3['ultimo_pagamento'] <= 75),
    (df3['ultimo_pagamento'] > 75)
    ]

# lista de valores que fazem match com as categorias de recencia
categorias = ["5", "4", "3", "2", "1"]

# criacao da coluna com as categorias correspondentes
df3['q_ult_pag'] = np.select(condicoes, categorias)

In [12]:
#testando código pra ver se funcionou direitinho
display(df3[df3['q_ult_pag'] != "5"])


,id_contrato,aging,ultimo_pagamento,receita,seq_meses,transacoes,adesao,q_ult_pag
9,19878149,44,80,341.95,5,5,outros,1
22,25650273,12,51,291.95,5,5,outros,3
46,28532934,4,35,149.97,3,3,outros,4
63,25777098,12,49,229.08,6,6,outros,3
93,20919074,34,34,383.94,6,6,outros,4
...,...,...,...,...,...,...,...,...
2266143,28687503,3,43,99.98,2,2,outros,4
2266156,28529989,4,66,99.98,2,2,outros,2
2266179,25401100,14,35,111.98,2,2,outros,4
2266217,28738139,3,35,99.98,2,2,outros,4


In [13]:
# Criando as categorias de valores de Frequência pra base toda

# lista de condicoes
condicoes = [
    (df3['transacoes'] <= 3),
    (df3['transacoes'] == 4),
    (df3['transacoes'] == 5),
    (df3['transacoes'] == 6),
    (df3['transacoes'] >= 7)
    ]

# lista de valores que fazem match com as categorias de recencia
categorias = ["1", "2", "3", "4", "5"]

# criacao da coluna com as categorias correspondentes
df3['q_trans'] = np.select(condicoes, categorias)

In [15]:
#Criando os quintis dos valores de RFV+A
df3['q_receita'] = pd.cut(df3['receita'], 5, labels=np.arange(1,6,1),right=False)
df3['q_aging'] = pd.cut(df3['aging'], 5, labels=np.arange(1,6,1),right=False)

df3['q_ult_pag'] = df3['q_ult_pag'].astype(int)
df3['q_trans'] = df3['q_trans'].astype(int)
df3['q_receita'] = df3['q_receita'].astype(int)
df3['q_aging'] = df3['q_aging'].astype(int)


#o transações contém muitos outliers, então pra contornar isso preferio usar esse código que calcula todos os valores outliers numa categoria só
#df3['q_trans'] = np.where(df3['transacoes'] >= df3['transacoes'].quantile(0.3), 5, pd.cut(df3['transacoes'], bins=5, labels=np.arange(1, 6, 1)))
#df3['q_trans'] = pd.qcut(df3['transacoes'], q=max(1, int(len(df3) * 0.05)), labels=False, duplicates='drop') + 1
display(df3)

,id_contrato,aging,ultimo_pagamento,receita,seq_meses,transacoes,adesao,q_ult_pag,q_trans,q2_ult_pag,q_receita,q_aging
0,12495686,84,8,555.93,7,7,outros,5,5,5,3,4
1,26132537,11,23,277.83,7,7,outros,5,5,5,2,1
2,26141204,11,29,179.34,6,6,outros,5,4,5,1,1
3,28546838,4,2,199.96,4,4,outros,5,2,5,1,1
5,18937705,51,27,413.94,6,6,outros,5,4,5,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2266282,28786817,2,27,119.98,2,2,outros,5,1,5,1,1
2266284,28780924,2,29,99.98,2,2,outros,5,1,5,1,1
2266289,28830525,2,23,99.98,2,2,outros,5,1,5,1,1
2266295,28774317,2,27,99.98,2,2,outros,5,1,5,1,1


In [16]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2136444 entries, 0 to 2266299
Data columns (total 12 columns):
 #   Column            Dtype   
---  ------            -----   
 0   id_contrato       int64   
 1   aging             int64   
 2   ultimo_pagamento  int64   
 3   receita           float64 
 4   seq_meses         int64   
 5   transacoes        int64   
 6   adesao            object  
 7   q_ult_pag         int64   
 8   q_trans           int64   
 9   q2_ult_pag        category
 10  q_receita         int64   
 11  q_aging           int64   
dtypes: category(1), float64(1), int64(9), object(1)
memory usage: 197.6+ MB


In [17]:
#Criando a variável de RFV e RFV+A
df3['RFV'] = df3['q_ult_pag'].astype(str)+df3['q_trans'].astype(str)+df3['q_receita'].astype(str)
df3['RFV+A'] = df3['q_ult_pag'].astype(str)+df3['q_trans'].astype(str)+df3['q_receita'].astype(str)+df3['q_aging'].astype(str)
display(df3)

,id_contrato,aging,ultimo_pagamento,receita,seq_meses,transacoes,adesao,q_ult_pag,q_trans,q2_ult_pag,q_receita,q_aging,RFV,RFV+A
0,12495686,84,8,555.93,7,7,outros,5,5,5,3,4,553,5534
1,26132537,11,23,277.83,7,7,outros,5,5,5,2,1,552,5521
2,26141204,11,29,179.34,6,6,outros,5,4,5,1,1,541,5411
3,28546838,4,2,199.96,4,4,outros,5,2,5,1,1,521,5211
5,18937705,51,27,413.94,6,6,outros,5,4,5,2,2,542,5422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266282,28786817,2,27,119.98,2,2,outros,5,1,5,1,1,511,5111
2266284,28780924,2,29,99.98,2,2,outros,5,1,5,1,1,511,5111
2266289,28830525,2,23,99.98,2,2,outros,5,1,5,1,1,511,5111
2266295,28774317,2,27,99.98,2,2,outros,5,1,5,1,1,511,5111


In [18]:
#Total de Registros na TIM Controle (N = 2262801)
#Total de Registros sem os Aderidos (n = 2136470)
df3.count()

id_contrato         2136444
aging               2136444
ultimo_pagamento    2136444
receita             2136444
seq_meses           2136444
transacoes          2136444
adesao              2136444
q_ult_pag           2136444
q_trans             2136444
q2_ult_pag          2136444
q_receita           2136444
q_aging             2136444
RFV                 2136444
RFV+A               2136444
dtype: int64

In [19]:
#Contabilização de quantos sujeitos tem nos grupos de ultimos pagamentos
df3.groupby('q_ult_pag')[['ultimo_pagamento']].describe().applymap('{0:.3f}'.format)


ultimo_pagamento                                                 \
                     count    mean    std     min     25%     50%     75%   
q_ult_pag                                                                   
1                36351.000  84.609  7.689  76.000  79.000  83.000  89.000   
2                34680.000  68.394  4.317  61.000  65.000  69.000  72.000   
3                37986.000  52.068  4.586  46.000  48.000  51.000  57.000   
4                79924.000  37.265  4.299  31.000  33.000  37.000  41.000   
5              1947503.000  15.400  8.447   0.000   8.000  15.000  22.000   

                    
               max  
q_ult_pag           
1          174.000  
2           75.000  
3           60.000  
4           45.000  
5           30.000

In [20]:
df3.groupby('q_trans')[['transacoes']].describe().applymap('{0:.3f}'.format)


transacoes                                                  
               count   mean    std    min    25%    50%    75%     max
q_trans                                                               
1         199120.000  2.486  0.500  2.000  2.000  2.000  3.000   3.000
2         109777.000  4.000  0.000  4.000  4.000  4.000  4.000   4.000
3         164418.000  5.000  0.000  5.000  5.000  5.000  5.000   5.000
4         581054.000  6.000  0.000  6.000  6.000  6.000  6.000   6.000
5        1082075.000  7.003  0.054  7.000  7.000  7.000  7.000  16.000

In [21]:
df3.groupby('q_receita')[['receita']].describe().applymap('{0:.3f}'.format)


receita                                                 \
                 count      mean     std      min       25%       50%   
q_receita                                                               
1           468139.000   169.847  45.889    0.000   144.240   173.400   
2          1467939.000   378.899  66.312  249.130   324.000   383.940   
3           166623.000   564.899  50.836  497.980   555.930   555.930   
4            32618.000   853.197  41.976  748.550   869.930   869.930   
5             1125.000  1101.260  46.397  999.920  1094.930  1094.930   

                               
                75%       max  
q_receita                      
1           203.960   248.970  
2           415.930   497.950  
3           555.930   745.930  
4           869.930   989.920  
5          1094.930  1244.920

In [22]:
df3.groupby('q_aging')[['aging']].describe().applymap('{0:.3f}'.format)


aging                                                      \
               count     mean    std      min      25%      50%      75%   
q_aging                                                                    
1        1415966.000   10.689  5.988    1.000    7.000   10.000   13.000   
2         322374.000   39.148  7.698   27.000   32.000   38.000   46.000   
3         192403.000   64.646  7.178   53.000   58.000   65.000   71.000   
4         155112.000   89.069  7.399   78.000   83.000   89.000   95.000   
5          50589.000  112.071  6.148  104.000  107.000  111.000  116.000   

                  
             max  
q_aging           
1         26.000  
2         52.000  
3         77.000  
4        103.000  
5        129.000

Separação das Categorias de RFV usando os valores min e max de cada grupo pela separação dos quintis:

Recência (dias)
R5 - 0 - 30 dias (n = 1,948,241): ≈ 91.19%
R4 - 31 - 45 (n = 82,735): ≈ 3.88% 
R3 - 46 - 60 (n = 34,006): ≈ 1.59% 
R2 - 61 - 75 (n = 38,742): ≈ 1.81% 
R1 - 76+ (n = 32,746): ≈ 1.53% 

Frequência (qtd transações)
F5 - >= 7 (n = 769,245): ≈ 36.00%
F4 - 6 (n = 851,236): ≈ 39.84%
F3 - 5 (n = 191,503): ≈ 8.96%
F2 - 4 (n = 122,318): ≈ 5.73%
F1 - <= 3 (n = 202,168): ≈ 9.47%

Valor (R$ em 6 meses)
V5 - R$974,93 - R$1199,93 (n = 1043): ≈ 0.05%
V4 - R$720,94 - R$953,74 (n = 39294): ≈ 1.84%
V3 - R$480,71 - R$719,94 (n = 187677): ≈ 8.78%
V2 - R$239,99 - R$479,97 (n = 1431645): ≈ 66.99%
V1 - R$0 - R$239,98 (n = 476811): ≈ 22.33%

Aging (em meses)
A5 - 104 - 129 (n = 49,453): ≈ 2.31%
A4 - 78 - 103 (n = 155,448): ≈ 7.27%
A3 - 53 - 77 (n = 192,417): ≈ 9.00%
A2 - 27 - 52 (n = 323,782): ≈ 15.14%
A1 - 1 - 26 (n = 1,415,370): ≈ 66.28%

Grupo Adesões #126331 ≈ 5.58%

Grupo to-Churn (df3[df3['q_ult_pag'] == 1]) #33083 ≈ 1.82%

Grupo Latão (df3[df3['q_ult_pag'] > 1 & < 5]) #152485 ≈ 8.40%

Grupo Bronze (df3[(df3['q_ult_pag'] == 5) & (df3['q_trans'] < 4)].count() #355746 ≈ 19.63%
 
Grupo Prata (df3[(df3['q_ult_pag'] == 5) & (df3['q_trans'] == 4)].count() #695460 ≈ 38.34%

Grupo Ouro (df3[(df3['q_ult_pag'] == 5) & (df3['q_trans'] == 5) & (df3['q_receita'] <= 2) & (df3['q_aging'] == 1)].count() #463447 ≈ 25.55%

Grupo Platina (df3[(df3['q_ult_pag'] == 5) & (df3['q_trans'] == 5) & (df3['q_receita'] >= 3) & (df3['q_aging'] >= 2)].count()  #117461 ≈ 6.48%



quantidade de transacoes para usuarios cujo aging < timeframe da analise

In [23]:
#Calculando a tabela de correlação entre as variáveis de RFV+A pra identificar se tem alguma que explica mais as outras
from scipy.stats import pearsonr
import numpy as np
rho = df3[['ultimo_pagamento', 'transacoes', 'receita', 'aging']].corr()
pval = df3[['ultimo_pagamento', 'transacoes', 'receita', 'aging']].corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=t]))
rho.round(2).astype(str) + p

,ultimo_pagamento,transacoes,receita,aging
ultimo_pagamento,1.0***,-0.36***,-0.23***,-0.06***
transacoes,-0.36***,1.0***,0.67***,0.37***
receita,-0.23***,0.67***,1.0***,0.62***
aging,-0.06***,0.37***,0.62***,1.0***


In [24]:
df3[(df3['q_ult_pag'] == 5) & (df3['q_trans'] == 5) & (df3['q_receita'] >= 3) & (df3['q_aging'] >= 2)].count() 

id_contrato         143086
aging               143086
ultimo_pagamento    143086
receita             143086
seq_meses           143086
transacoes          143086
adesao              143086
q_ult_pag           143086
q_trans             143086
q2_ult_pag          143086
q_receita           143086
q_aging             143086
RFV                 143086
RFV+A               143086
dtype: int64

In [25]:
df3.groupby('RFV+A_bin')[['ultimo_pagamento','transacoes','receita','aging']].describe().applymap('{0:.3f}'.format)


KeyError: 'RFV+A_bin'

In [ ]:
#Separação novamente em 5 grupos usando os quintis em cima dos valores de RFV+A
#df3['grupo'] = pd.cut(df3['RFV+A'].astype(int), 7, labels=False)
#df3.describe().apply(lambda s: s.apply('{0:.3f}'.format))

In [ ]:
#df4 = df3[(df3["RFV+A"].astype(int) >= 5555) & (df3["RFV+A"].astype(int) >= 4444)] #432849
#df4 = df3[(df3["RFV+A"].astype(int) <= 4444) & (df3["RFV+A"].astype(int) >= 3333)] #625667
#df4 = df3[(df3["RFV+A"].astype(int) <= 3333)] #& (df3["RFV+A"].astype(int) >= 2222)] #1005885
#df4 = df3[(df3["RFV+A"].astype(int) <= 2222) & (df3["RFV+A"].astype(int) >= 1111)] #398449

In [ ]:
#df3['q2_ult_pag'] = pd.qcut(df3['ultimo_pagamento'], 5, labels=np.arange(5,0,-1))
#df3[['ultimo_pagamento']].groupby(df3['q2_ult_pag']).describe().applymap('{0:.3f}'.format)
#df3['q2_ult_pag'] = pd.cut(df3['ultimo_pagamento'], 5, labels=np.arange(5,0,-1))
#df3[['ultimo_pagamento']].groupby(df3['q2_ult_pag']).describe().applymap('{0:.3f}'.format)